In [1]:
from fastai import *
from fastai.tabular import *
from utils import *
import numpy as np
import dask.dataframe as dd

In [2]:
torch.cuda.is_available()

True

In [5]:
path = '/home/shanmugam/fastai/m5'
ddf = pd.read_csv(f'{path}/sales_train_validation.csv')
calendar = pd.read_csv(f'{path}/calendar.csv', parse_dates = ['date'])
sell_prices = pd.read_csv(f'{path}/sell_prices.csv')
submission = pd.read_csv(f'{path}/sample_submission.csv')

In [6]:
sale_train_val = dd.from_pandas(ddf)

ValueError: Exactly one of npartitions and chunksize must be specified.

In [7]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                   'y': [1., 2., 3., 4., 5.]})
df

,x,y
0,1,1.0
1,2,2.0
2,3,3.0
3,4,4.0
4,5,5.0


In [4]:
sale_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [5]:
calendar[:10]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [6]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [7]:
# Get all columns that aren't day columns
list_id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] 
df_d_cols = sale_train_val.drop(list_id_vars, axis=1)
sale_train_val.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1919)

In [8]:
%time df_melted_sales = sale_train_val.melt(id_vars = list_id_vars, value_vars = df_d_cols.columns, var_name = 'd', value_name = 'sales') 

CPU times: user 11.5 s, sys: 1.43 s, total: 12.9 s
Wall time: 12.1 s


In [9]:
df_melted_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3


In [10]:
df_melted_sales.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1, inplace = True) 
df_melted_sales.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1,0
1,HOBBIES_1_002_CA_1_validation,d_1,0
2,HOBBIES_1_003_CA_1_validation,d_1,0
3,HOBBIES_1_004_CA_1_validation,d_1,0
4,HOBBIES_1_005_CA_1_validation,d_1,0


In [11]:
sub_cols = submission.drop(['id'], axis=1).columns

# Like what we did to the sales data, we also need to melt the submission 
df_melted_sub = submission.melt(id_vars = ['id'], value_vars = sub_cols, 
                                    var_name = 'd', value_name = 'sales') 
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,F1,0
1,HOBBIES_1_002_CA_1_validation,F1,0
2,HOBBIES_1_003_CA_1_validation,F1,0
3,HOBBIES_1_004_CA_1_validation,F1,0
4,HOBBIES_1_005_CA_1_validation,F1,0


In [12]:
df_melted_sub['d'] = df_melted_sub['d'].str.replace('F','')
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1,0
1,HOBBIES_1_002_CA_1_validation,1,0
2,HOBBIES_1_003_CA_1_validation,1,0
3,HOBBIES_1_004_CA_1_validation,1,0
4,HOBBIES_1_005_CA_1_validation,1,0


In [13]:
df_melted_sub['d'] = pd.to_numeric(df_melted_sub['d'], errors='coerce') 
df_melted_sub['d'] = df_melted_sub['d'] + 1913
df_melted_sub = df_melted_sub.applymap(str)
df_melted_sub['d'] = 'd_'+ df_melted_sub['d'].astype(str)
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1914,0
1,HOBBIES_1_002_CA_1_validation,d_1914,0
2,HOBBIES_1_003_CA_1_validation,d_1914,0
3,HOBBIES_1_004_CA_1_validation,d_1914,0
4,HOBBIES_1_005_CA_1_validation,d_1914,0


In [14]:
df_melted_sub.tail() 

,id,d,sales
1707435,FOODS_3_823_WI_3_evaluation,d_1941,0
1707436,FOODS_3_824_WI_3_evaluation,d_1941,0
1707437,FOODS_3_825_WI_3_evaluation,d_1941,0
1707438,FOODS_3_826_WI_3_evaluation,d_1941,0
1707439,FOODS_3_827_WI_3_evaluation,d_1941,0


In [15]:
%who DataFrame

calendar	 df_d_cols	 df_melted_sales	 df_melted_sub	 sale_train_val	 sell_prices	 submission	 


In [16]:
del sale_train_val
del submission
del df_d_cols
gc.collect()

67

In [17]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))

In [18]:
reduce_mem_usage(calendar)
reduce_mem_usage(df_melted_sales)
reduce_mem_usage(df_melted_sub) 
reduce_mem_usage(sell_prices)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 1001.26 Mb (25.0% reduction)
Mem. usage decreased to 39.08 Mb (0.0% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [19]:
df_melted_sales['d'].value_counts()

d_826     30490
d_1499    30490
d_615     30490
d_470     30490
d_1383    30490
          ...  
d_1316    30490
d_567     30490
d_1609    30490
d_1823    30490
d_512     30490
Name: d, Length: 1913, dtype: int64

In [20]:
df_test = df_melted_sales.tail(853720) 
df_train = df_melted_sales.iloc[:-853720] 
df_submission = df_melted_sub.copy()

In [21]:
df_train.shape, df_test.shape, df_submission.shape

((57473650, 3), (853720, 3), (1707440, 3))

In [22]:
df_train.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1,0
1,HOBBIES_1_002_CA_1_validation,d_1,0
2,HOBBIES_1_003_CA_1_validation,d_1,0
3,HOBBIES_1_004_CA_1_validation,d_1,0
4,HOBBIES_1_005_CA_1_validation,d_1,0


In [23]:
# Merge the calendar data with the training, test and submission DFs
df_train = df_train.merge(calendar, left_on='d', right_on='d', how='left')
df_test = df_test.merge(calendar, left_on='d', right_on='d', how='left')
df_submission = df_submission.merge(calendar, left_on='d', right_on='d', how='left')
df_train.shape, df_test.shape#, df_submission.shape # should be same numb of cols

((57473650, 16), (853720, 16))

In [24]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [25]:
sell_prices['id'] = sell_prices['item_id'] +'_' + sell_prices['store_id']
sell_prices.drop(['item_id', 'store_id'], axis=1, inplace = True)
sell_prices.head()

,wm_yr_wk,sell_price,id
0,11325,9.578125,HOBBIES_1_001_CA_1
1,11326,9.578125,HOBBIES_1_001_CA_1
2,11327,8.257812,HOBBIES_1_001_CA_1
3,11328,8.257812,HOBBIES_1_001_CA_1
4,11329,8.257812,HOBBIES_1_001_CA_1


In [26]:
df_train.shape

(57473650, 16)

In [27]:
df_train['id_for_price'] = df_train['id']

In [ ]:
level_map = {'_validation': ''}
df_train['id_for_price'] =  df_train['id_for_price'].map(level_map)

In [29]:
df_train.head()

,id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,id_for_price
0,HOBBIES_1_001_CA_1_validation,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_001_CA_1_validation
1,HOBBIES_1_002_CA_1_validation,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_002_CA_1_validation
2,HOBBIES_1_003_CA_1_validation,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_003_CA_1_validation
3,HOBBIES_1_004_CA_1_validation,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_004_CA_1_validation
4,HOBBIES_1_005_CA_1_validation,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,HOBBIES_1_005_CA_1_validation


In [ ]:
del(df_melted_sales)
del(df_melted_sub)
gc.collect()

In [ ]:
df_test['id_for_price'] = df_test['id'].str.replace('_validation','')
df_submission['id_for_price'] = df_submission['id'].str.replace('_evaluation','')
df_submission['id_for_price'] = df_submission['id_for_price'].str.replace('_validation','')

In [ ]:
sell_prices = sell_prices.sort_values(by=['wm_yr_wk', 'id'])

In [ ]:
sell_prices

In [ ]:
df_train 

In [ ]:
df_train.set_index(['id_for_price', 'wm_yr_wk'], inplace = True)
sell_prices.set_index(['id', 'wm_yr_wk'], inplace = True)
df_train = df_train.join(sell_prices, how='left')

In [39]:
df_train = pd.merge(df_train, sell_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], 
                    right_on = ['id', 'wm_yr_wk'])

ValueError: 'id' is both an index level and a column label, which is ambiguous.

In [ ]:
df_test = pd.merge(df_test, sell_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], 
                   right_on = ['id', 'wm_yr_wk'])
df_submission = pd.merge(submission, sell_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], 
                         right_on = ['id', 'wm_yr_wk'])

# Check to see if there are any missing prices
df_train['sell_price'].count(), df_train.shape, df_test['sell_price'].count(), df_test.shape, df_submission['sell_price'].count(), df_submission.shape

In [ ]:
df_train.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_train.rename(columns = {"id_x":"id"}, inplace = True)
df_test.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_test.rename(columns = {"id_x":"id"}, inplace = True)
df_submission.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_submission.rename(columns = {"id_x":"id"}, inplace = True)